# Parameters

In [1]:
num_of_epochs = 5
batch_size = 32
dataset_name = 'plantVillage-test-mohanty'
dataset_path = '../datasets/' + dataset_name
model_diagram_path = 'alexnet_model_diagram.png'
model_save_path = 'alexnet_' + dataset_name +'.h5'
plot_save_path = 'plot.png'


# Load dataset

In [2]:
import sys, os
sys.path.append('..')
import numpy as np
from imutils import paths



from utils.preprocessors.aspect_aware_preprocessor import AspectAwarePreprocessor
from utils.preprocessors.image_to_array_preprocessor import ImageToArrayPreprocessor
from utils.io.simple_dataset_loader import SimpleDatasetLoader

# Get the list of image names
image_paths = list(paths.list_images(dataset_path))

print(len(image_paths), 'images loaded')

# Get unique class_names
class_names = [pt.split(os.path.sep)[-2] for pt in image_paths]
class_names = [str(x) for x in np.unique(class_names)]
print(class_names)


# Preprocessors
aap = AspectAwarePreprocessor(227, 227)
itap = ImageToArrayPreprocessor()

# Add preprocessors to DataSetLoader class
sdl = SimpleDatasetLoader(preprocessors=[aap, itap])

(data, labels) = sdl.load(image_paths, verbose=500)
data = data.astype('float') / 255.0



Using TensorFlow backend.
1458 images loaded
['Tomato___Bacterial_spot', 'Tomato___Early_blight', 'Tomato___healthy']
[INFO]: Processed 500/1458
[INFO]: Processed 1000/1458


# Encode labels and split dataset

In [3]:
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split

# Split data into training (75%) and testing (25%) data
(train_x, test_x, train_y, test_y) = train_test_split(data, labels,
test_size=0.25, random_state=0)

# Convert the labels from integers to vectors
lb = LabelBinarizer()
train_y = lb.fit_transform(train_y)
test_y = lb.transform(test_y)

# Initialize model

In [5]:
import os, sys
# import pydot_ng as pydot
# pydot.find_graphviz()
# os.environ.get('PATH', '')

# remove these
# sys.path.append('..')
# class_names = ['one', 'two', 'three', 'four']
# remove these

from utils.models.alexnet import alexnet
from keras.utils.vis_utils import plot_model
from keras.optimizers import SGD

model = alexnet(len(class_names))
model.summary()

# plot_model(model, to_file=model_diagram_path, show_shapes=True)

# copile model
opt = SGD(lr=0.05)
model.compile(loss="categorical_crossentropy", optimizer=opt,
              metrics=["accuracy"])



Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 55, 55, 96)        34944     
_________________________________________________________________
activation_1 (Activation)    (None, 55, 55, 96)        0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 27, 27, 96)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 17, 17, 256)       2973952   
_________________________________________________________________
activation_2 (Activation)    (None, 17, 17, 256)       0         
_________________________________________________________________
max_pooling2d_2 (MaxP

# Fit model

In [6]:
# train the network
print("[INFO] training network...")
H = model.fit(train_x, train_y, validation_data=(test_x, test_y),
              batch_size=batch_size, epochs=num_of_epochs, verbose=1)

model.save(model_save_path)

[INFO] training network...
Instructions for updating:
Use tf.cast instead.
Train on 1093 samples, validate on 365 samples
Epoch 1/5
1093/1093 [==============================] - 113s 103ms/step - loss: 1.0642 - acc: 0.4886 - val_loss: 1.0496 - val_acc: 0.4767
Epoch 2/5
1093/1093 [==============================] - 103s 94ms/step - loss: 1.0407 - acc: 0.4904 - val_loss: 1.0361 - val_acc: 0.4767
Epoch 3/5
1093/1093 [==============================] - 99s 91ms/step - loss: 1.0151 - acc: 0.5215 - val_loss: 1.3148 - val_acc: 0.2959
Epoch 4/5
1093/1093 [==============================] - 99s 91ms/step - loss: 1.0562 - acc: 0.4858 - val_loss: 1.0368 - val_acc: 0.4767
Epoch 5/5
1093/1093 [==============================] - 99s 91ms/step - loss: 1.0155 - acc: 0.5050 - val_loss: 1.6798 - val_acc: 0.2959


# Generate classfication report

In [7]:
from sklearn.metrics import classification_report


# If one hot encoding is used
predictions = model.predict(test_x, batch_size=batch_size)
print(classification_report(test_y.argmax(axis=1), predictions.argmax(axis=1), target_names=class_names))


precision    recall  f1-score   support

Tomato___Bacterial_spot       0.00      0.00      0.00       174
  Tomato___Early_blight       0.00      0.00      0.00        83
       Tomato___healthy       0.30      1.00      0.46       108

               accuracy                           0.30       365
              macro avg       0.10      0.33      0.15       365
           weighted avg       0.09      0.30      0.14       365

/Users/praveenmuthukumarana/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


# Plot model

In [8]:
print(H.history.keys())

print('loss')
print(H.history['loss'])
print('')

print('acc')
print(H.history['acc'])
print('')


print('val_loss')
print(H.history['val_loss'])
print('')

print('val_acc')
print(H.history['val_acc'])
print('')

dict_keys(['val_loss', 'val_acc', 'loss', 'acc'])
loss
[1.0641913340927152, 1.0406997189648306, 1.01506256490046, 1.0562494123952715, 1.0154840067669764]

acc
[0.48856358647291964, 0.4903934127348667, 0.5215004574701749, 0.48581884723678165, 0.505032022012447]

val_loss
[1.0495979145781635, 1.036134089182501, 1.3147825139842622, 1.0367660956839992, 1.6798063085503774]

val_acc
[0.47671232917537426, 0.47671232917537426, 0.2958904110405543, 0.47671232917537426, 0.2958904110405543]



In [9]:
# plot the training loss and accuracy
from matplotlib import pyplot as plt
plt.style.use("ggplot")
plt.figure()
plt.plot(np.arange(0, num_of_epochs), H.history["loss"], label="train_loss")
plt.plot(np.arange(0, num_of_epochs), H.history["val_loss"], label="val_loss")
plt.plot(np.arange(0, num_of_epochs), H.history["acc"], label="train_acc")
plt.plot(np.arange(0, num_of_epochs), H.history["val_acc"], label="val_acc")
plt.title("Training Loss and Accuracy")
plt.xlabel("Epoch #")
plt.ylabel("Loss/Accuracy")
plt.legend()
plt.savefig(plot_save_path)
plt.show()

<Figure size 640x480 with 1 Axes>